# <font color='purple'>CDC with WordCloud</font>

By: Ameer Syedibrahim

### Import Statements

In [10]:
import pyexcel as pe
import pyexcel.ext.xls, time, string, nltk
import numpy as np
from nltk import word_tokenize
from collections import Counter
from functools import partial
from operator import ne

### Loading in Chat Data Excel File

In [11]:
your_matrix = np.array(pe.get_array(file_name="Customer_Only.csv"))

### Guidelines for Labeling
Currently labeling is done with key words that were manually chosen. This approach can be refined with the two methods shown below

In [12]:

end_service_list = ['moving','end','service','action','Verizon','year','agreement','cancel','account','online']
difficulty_paying_online_list = ['difficulty','pay','online','bill','check','trouble','tough','transaction','hard','time','account']
keep_internet_only_list = ['internet','only','keep','just', 'cancel','faster','stronger','fast','strong','connection','wifi']
router_problems_list = ['router','issue','problem','switch','does','need','connect','network','restart','lost','option','help', 'technician','error']
reduce_bill_list = ['reduce','bill','pay','currently','how','cheap','cheaper','discount','afford','affordable','expensive','lowered','lower','low']

labels_dict = {}
labels_dict["END SERVICE"] = end_service_list
labels_dict["DIFFICULTY PAYING ONLINE"] = difficulty_paying_online_list
labels_dict["KEEP INTERNET ONLY"] = keep_internet_only_list 
labels_dict["ROUTER PROBLEMS"] = router_problems_list
labels_dict["REDUCE CUSTOMER\'S BILL"] = reduce_bill_list 



In [13]:
credit_check_list = []
technician_list = []
freeze_list = ["system","stuck","freeze","unable","couldn't","stops"]

### Tokenization of Input Text
The input text is tokenized into a list with all stop words and other irrelevant words removed

In [14]:
def tokenize(text, preserve_case=True, filter_text=True, stopwords=None, punc=string.punctuation):
    if filter_text and stopwords is None:
        with open("stopwords.txt", 'r') as r:
            stops = []
            for line in r:
                stops += [i.strip() for i in line.split('\t')]
    stopwords = stops
    tokens = word_tokenize(text)
    indices = []
    if filter_text:
        for i in range(len(tokens)):
            word = tokens[i]
            # print(word, word in punc, word.lower() in stopwords)
            if word in punc or word.lower() in stopwords:
                indices.append(i)
        tokens = list(np.delete(np.array(tokens), np.array(indices)))
    if preserve_case:
        return tokens
    return [i.lower() for i in a]

### Finding Common Words
The common words in a list of inputted tokens and a list of tokens which represents words that belong to a label
are determined

In [18]:
def list_comm(tokens, label_tokens):

    return list(set(tokens).intersection(label_tokens))

### Assigning a Label
If the string input has four or more common words with a list of words that correspond to a label, then that label is returned

In [19]:
def assign_labels(string):
    
    tokens = tokenize(string)

    for label in labels_dict:
        
        intersection = list_comm(tokens,labels_dict[label])
        
        if len(intersection) >= 4:
            
            return label

### Executing Label Classification

In [20]:
def execute():
    
    string = input()
    
    print("  ")
    print("LABEL: ",assign_labels(string))
#############################################################################################################################
def execute_labels(string):
    
    
    print("  ")
    print("LABEL: ",assign_labels(string))

## <font color='blue'>Method 1: Selectively Counting Word Occurrence</font>


### Guidelines for Labeling (Method 1)
Keywords for labels are chosen based on analyzing certain portions of the Chat Data, as described above with this Method

In [21]:
internet_issues1 = ["speed",]
keep_one_or_more_services1 = []
router_issues1= []
placing_order_issues1 = []
customer_has_bill_increase = []

labels_dict1 = {}
labels_dict1["INTERNET ISSUES"] = end_service_list1
labels_dict1["KEEP ONE OR MORE SERVICES"] = difficulty_paying_online_list1
labels_dict1["ROUTER ISSUES"] = keep_internet_only_list1 
labels_dict1["PLACING ORDER ISSUES"] = router_problems_list1
labels_dict1["CUSTOMER HAS BILL INCREASE"] = reduce_bill_list1 

NameError: name 'end_service_list1' is not defined

### Assigning Labels with Method 1

In [22]:
def assign_labels1(string):
    
    tokens = tokenize(string)

    for label in labels_dict1:
        
        intersection = list_comm(tokens,labels_dict1[label])
        
        if len(intersection) >= 4:
            
            return label

## <font color='green'>Method 2: Counting Word Occurrence of Entire Data</font>
Feeding in one long list of tokens corresponding to the all the data that is fed in

### Developing a List of token lists
For each string in the matrix, the relevant words are tokenized into a list and appended to a list. This process takes approximately 30 seconds for one day's worth of data

In [23]:
the_list = []
t0 = time.time()

for i in range(your_matrix.shape[0]):
    
    tokens = tokenize(your_matrix[i][0])
    
    if i == 0:
        
        the_list = tokens
        
    else:
        the_list.extend(tokens)
        
t1 = time.time()
print("Time Elapsed: " + str(t1 - t0) + " s")

C:\Users\syediam\AppData\Local\Cisco\New\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  app.launch_new_instance()


Time Elapsed: 44.41814351081848 s


### Counting Word Occurrence
Each occurrence of a word is recorded and tallied up to determine the total occurrences of a particular word in a list of data. After stopwords and other irrelevant words are filtered out, the 20 most common words are displayed

In [24]:
the_list = list(filter(partial(ne, "connectivityThe"), the_list))
the_list = list(filter(partial(ne, "youCustomer"), the_list))
the_list = list(filter(partial(ne, "connectionThe"), the_list))
the_list = list(filter(partial(ne, "connectionI"), the_list))
the_list = list(filter(partial(ne, "'s"), the_list))
the_list = list(filter(partial(ne, "connectionCustomer"), the_list))
the_list = list(filter(partial(ne, "connectionOkCustomer"), the_list))
the_list = list(filter(partial(ne, "okCustomer"), the_list))
the_list = list(filter(partial(ne, "..."), the_list))
the_list = list(filter(partial(ne, "'m"), the_list))
the_list = list(filter(partial(ne, "Verizon"), the_list))


counts = Counter(list(the_list))

the_top20, counts = list(zip(*counts.most_common(20)))

print("Twenty Most Common Words: ",the_top20)

Twenty Most Common Words:  ('internet', 'service', 'phone', 'number', 'account', 'bill', 'connectivity', 'minutes', 'plan', 'router', 'tv', 'order', 'box', 'month', 'call', 'set', 'change', 'home', 'TV', 'pay')


### Guidelines for Labeling (Method 2)
Keywords for labels are chosen based on analyzing all the Chat Data, as described above with this Method

In [25]:
end_service_list2 = []
difficulty_paying_online_list2 = []
keep_internet_only_list2 = []
router_problems_list2 = []
reduce_bill_list2 = []

labels_dict2 = {}
labels_dict2["END SERVICE"] = end_service_list2
labels_dict2["DIFFICULTY PAYING ONLINE"] = difficulty_paying_online_list2
labels_dict2["KEEP INTERNET ONLY"] = keep_internet_only_list2 
labels_dict2["ROUTER PROBLEMS"] = router_problems_list2
labels_dict2["REDUCE CUSTOMER\'S BILL"] = reduce_bill_list2 

### Assigning Labels with Method 2

In [26]:
def assign_labels2(string):
    
    tokens = tokenize(string)

    for label in labels_dict2:
        
        intersection = list_comm(tokens,labels_dict2[label])
        
        if len(intersection) >= 4:
            
            return label

### Final Classification
Classification for one day's worth of data takes approximately 40 seconds to complete

In [28]:
t0 = time.time()

for i in range(your_matrix.shape[0]):
    
    print(assign_labels(your_matrix[i][0]) )
    
    
t1 = time.time()

print("Time Elapsed: " + str(t1 - t0) + " s")

None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
KEEP INTERNET ONLY
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
ROUTER PROBLEMS
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


C:\Users\syediam\AppData\Local\Cisco\New\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  app.launch_new_instance()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
KEEP INTERNET ONLY
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

None
None
None
None
None
None
None
None
END SERVICE
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
No

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
END SERVICE
END SERVICE
END SERVICE
END SERVICE
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
DIFFICULTY PAYING ONLINE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
END SERVICE
END SERVICE
END SERVICE
END SERVICE
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
END SERVICE
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None